In [19]:
import sevenbridges as sbg
from sevenbridges.errors import SbgError
from sevenbridges.http.error_handlers import rate_limit_sleeper, maintenance_sleeper
import sys
import re
import pdb
import concurrent.futures
from requests import request
import datetime
config = sbg.Config(profile='turbo')
api = sbg.Api(config=config, error_handlers=[rate_limit_sleeper, maintenance_sleeper])

In [20]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

### get cost

In [ ]:
def parse_task(prefix, id_prefix, suffix, task):
    # pdb.set_trace()
    try:
        if re.search(prefix, task.name):
            # print (task.id + '\t' + task.name + '\t' + str(task.price.amount) + '\n')
            return task.id + '\t' + task.name + '\t' + str(task.price.amount) + '\t' + task.app + '\n'
    except Exception as e:
        sys.stderr.write('Could not process task ' + task.id + ' ' + task.name + ' got error ' + str(e) + '\n')
        sys.exit()

In [ ]:
pre = 'RNAseq-BS_'
id_pre = 'BS_'
suffix = ''
project = 'kids-first-drc/kids-first-drc-rnaseq-workflow'

tasks = api.tasks.query(project=project, status='COMPLETED').all()
#task = next(tasks)
#parse_task(pre, id_pre, suffix, task)
#pdb.set_trace()
out_fh = open('/Users/brownm28/Documents/2022-Feb-24_gtex_gencode_compare/EXPORT/STAR_pipeline_cost.txt', 'w')

with concurrent.futures.ThreadPoolExecutor(16) as executor:
    results = {executor.submit(parse_task, pre, id_pre, suffix, task): task for task in tasks}
    for cost_info in concurrent.futures.as_completed(results):
        try:
            if cost_info.result() is not None:
                out_fh.write(cost_info.result())
        except Exception as e:
            print (e)
out_fh.close()


### get exact run time

In [ ]:
def merge_and_sum_time_intervals(prefix, id_prefix, suffix, task):
    data = []
    if re.search(prefix, task.name):
        for job in task.get_execution_details().jobs:
            try:
                pair = (job.start_time, job.end_time)
                data.append(pair)
            except Exception as e:
                print (e)
                pdb.set_trace()
                hold = 1
        data = sorted(data, key=lambda x: x[0])


        # from https://stackoverflow.com/questions/34797525/how-to-correctly-merge-overlapping-datetime-ranges-in-python
        result = []
        try:
            t_old = data[0]
            for t in data[1:]:
                if t_old[1] >= t[0]:  #I assume that the data is sorted already
                    t_old = ((min(t_old[0], t[0]), max(t_old[1], t[1])))
                else:
                    result.append(t_old)
                    t_old = t

            else:
                result.append(t_old)
        except Exception as e:
            print(e)
            pdb.set_trace()
            hold = 1
        total_seconds = 0
        for t_int in result:
            total_seconds += (t_int[1] - t_int[0]).seconds
        #print ('Task ran in ' + str(total_seconds) + ', which is ' + str(total_seconds/3600) + ' hours')
        return task.id + '\t' + task.name + '\t' + str(total_seconds/3600) + '\n'
    

In [ ]:
# use task.get_execution_details().jobs - is a list that will giev you start and end time of each job, can use to create opverlapping tim e intervals, then bin and sum
pre = 'RNAseq-BS_'
id_pre = 'BS_'
#suffix = 'INPUT'
project = 'kids-first-drc/kids-first-drc-rnaseq-workflow'

tasks = api.tasks.query(project=project, status='COMPLETED').all()
# task = next(tasks)
# merge_and_sum_time_intervals(task)
out_fh = open('/Users/brownm28/Documents/2022-Feb-24_gtex_gencode_compare/EXPORT/star_run.txt', 'w')

with concurrent.futures.ThreadPoolExecutor(16) as executor:
    results = {executor.submit(merge_and_sum_time_intervals, pre, id_pre, suffix, task): task for task in tasks}
    for time_info in concurrent.futures.as_completed(results):
        try:
            if time_info.result() is not None:
                out_fh.write(time_info.result())
        except Exception as e:
            print (e)
out_fh.close()


In [41]:
task = api.tasks.get('aff86016-55b6-4c9b-8bed-6bf63cd526d6')
pdb.set_trace()
hold=1


--Return--
None
> <ipython-input-41-0b9f4a6920ec>(2)<module>()
      1 task = api.tasks.get('aff86016-55b6-4c9b-8bed-6bf63cd526d6')
----> 2 pdb.set_trace()
      3 hold=1

ipdb> task.jobs
*** AttributeError: 'Task' object has no attribute 'jobs'
ipdb> dir(task)
['_API', '_URL', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_api', '_data', '_dirty', '_fields', '_modified_data', '_old', '_query', '_serialize_execution_settings', '_serialize_inputs', '_set', '_to_api_file_format', '_update_read_only', 'abort', 'app', 'batch', 'batch_by', 'batch_group', 'batch_input', 'bulk_get', 'clone', 'create', 'created_by', 'created_time', 'deepcopy', 'delete', 'description', 'end_time', 'equals', 'errors', 

ipdb> task.get_execution_details().jobs[10]
<Job: name=run_lancet_lancet_21_s, status=COMPLETED>
ipdb> dir(task.get_execution_details().jobs[10])
['_API', '_URL', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_api', '_data', '_dirty', '_fields', '_modified_data', '_old', '_query', '_set', '_update_read_only', 'command_line', 'deepcopy', 'delete', 'docker', 'end_time', 'equals', 'field', 'get', 'instance', 'logs', 'name', 'reload', 'retried', 'start_time', 'status', 'update_old']
ipdb> q


BdbQuit: 

## Get Per-job-per-task run time and instance time

In [39]:
def get_instance_run_times(task):
    print("Processing task " + task.name)
    data = {}
    vm_types = {}
    for job in task.get_execution_details().jobs:
        if job.status == "COMPLETED" and job.instance.type != 'precomputed':
            try:
                # print (job.name + "\t" + str((job.end_time - job.start_time).seconds/3600))
                # sys.stderr.write(job.name + '\n')
                vm = job.instance.id
                if vm not in data:
                    data[vm] = []
                    vm_types[vm] = job.instance.type
                pair = (job.start_time, job.end_time)
                data[vm].append(pair)
            except Exception as e:
                print (e)
                pdb.set_trace()
                hold = 1
    total_hrs = 0
    total_spot_cost = 0
    total_direct_cost = 0
    machine_hrs = {}
#     c5_9_spot = 0.5164
#     c5_9_direct = 1.53
#     m4_10_spot = 0.412
#     m4_10_direct = 2
#     spot_rate = c5_9_spot
#     direct_rate = c5_9_direct
    print("Processing vm data")
    for vm in data:
        data[vm] = sorted(data[vm], key=lambda x: x[0])
        result = []
        try:
            t_old = data[vm][0]
            for t in data[vm][1:]:
                if t_old[1] >= t[0]:  #I assume that the data is sorted already
                    t_old = ((min(t_old[0], t[0]), max(t_old[1], t[1])))
                else:
                    result.append(t_old)
                    t_old = t

            else:
                result.append(t_old)
        except Exception as e:
            print(e)
            pdb.set_trace()
            hold = 1
        total_seconds = 0
        for t_int in result:
            total_seconds += (t_int[1] - t_int[0]).seconds
        #print ('Task ran in ' + str(total_seconds) + ', which is ' + str(total_seconds/3600) + ' hours')
        # print(vm + '\t' + '\t' + str(total_seconds/3600))
        run_hrs = total_seconds/3600
        vm_type = vm_types[vm]
        if vm_type not in machine_hrs:
            machine_hrs[vm_type] = 0
        spot_cost = run_hrs * spot_instance[vm_type]
        direct_cost = run_hrs * on_demand_instance[vm_type]
        out_fh.write("\t".join([task.name, task.id, vm_type, vm, str(run_hrs), str(spot_cost), str(direct_cost)]) + "\n")
        total_hrs += run_hrs
        machine_hrs[vm_type] += run_hrs
        total_spot_cost += spot_cost 
        total_direct_cost += direct_cost
    
    summary.write("\t".join([task.name, task.id, str(task.price.amount), str(total_spot_cost), str(total_direct_cost)]) + '\n')
    for vm_type in machine_hrs:
        machine_hrs_fh.write("\t".join([task.name, task.id,vm_type, str(machine_hrs[vm_type])]) + '\n')

In [44]:
pre = 'Kids First DRC Somatic Variant Workflow'
#suffix = 'INPUT'
project = 'kfdrc-harmonization/sd-bhjxbdqk-x01-somatic-mutations-wgs-tumor'
# bs_ids = ['BS_SR0TJ11R','BS_P9P1N48D','BS_X6NW8R6K']
bs_ids = []
spot_instance= { 'r5.large': 0.0227, 'r5.4xlarge': 0.1867,'c4.2xlarge': 0.1896, 'c5.4xlarge': 0.2901, 'c5.9xlarge': 0.6689, 'c4.8xlarge': 0.5653, 'r5.2xlarge': 0.1795, 'c5.2xlarge': 0.1684, 'm5.2xlarge': 0.1405,'m5.xlarge': 0.0686, 'r5.xlarge': 0.0718}
on_demand_instance = { 'r5.large': 0.126,'r5.4xlarge': 1.008, 'c4.2xlarge': 0.398, 'c5.4xlarge': 0.68, 'c5.9xlarge': 1.53, 'c4.8xlarge': 1.591, 'r5.2xlarge': 0.504, 'c5.2xlarge': 0.34, 'm5.2xlarge': 0.384, 'm5.xlarge': 0.192, 'r5.xlarge': 0.252}

tasks = api.tasks.query(project=project, status='COMPLETED').all()
# task = next(tasks)
# merge_and_sum_time_intervals(task)
out_fh = open('/Users/brownm28/Documents/playground/somatic_cost_estimates/to_date_wf_est.txt', 'w')
out_fh.write('task_name\ttask_id\tinstance_type\tinstance_id\ttotal_run_time\tspot_cost\tdirect_cost\n')
summary = open('/Users/brownm28/Documents/playground/somatic_cost_estimates/to_date_wf_summary_est.txt', 'w')
summary.write('task_name\ttask_id\tactual_cost\tpure_spot_cost\tpure_direct_cost\n')
machine_hrs_fh = open('/Users/brownm28/Documents/playground/somatic_cost_estimates/to_date_wf_machine_hrs.txt', 'w')
machine_hrs_fh.write('task_name\ttask_id\tmachine_type\ttotal_run_hrs\n')
for task in tasks:
#     task_split = task.name.split(' - ')
    if re.search(pre, task.name):
        if bs_ids:
            bs_id = re.match('.*(BS_\w+) -vs-.*', task.name)
            if bs_id is not None and bs_id.group(1) in bs_ids:
                get_instance_run_times(task)
        else:
            get_instance_run_times(task)
out_fh.close()
summary.close()
machine_hrs_fh.close()


Processing task Kids First DRC Somatic Variant Workflow run - BS_420TZWST -vs- BS_EZA1EYWV
Processing vm data
Processing task Kids First DRC Somatic Variant Workflow run - BS_DM0358AQ -vs- BS_86TFE982
Processing vm data
Processing task Kids First DRC Somatic Variant Workflow run - BS_KVC8X5BX -vs- BS_2DAN8S59
Processing vm data
Processing task Kids First DRC Somatic Variant Workflow run - BS_B5T9CFX0 -vs- BS_8CBDRARJ
Processing vm data
Processing task Kids First DRC Somatic Variant Workflow run - BS_5357BVCQ -vs- BS_3VMWCBV7
Processing vm data
Processing task Kids First DRC Somatic Variant Workflow run - BS_FXWGAQQ1 -vs- BS_8CBDRARJ
Processing vm data
Processing task Kids First DRC Somatic Variant Workflow run - BS_BYNGYA6A -vs- BS_3VMWCBV7
Processing vm data
Processing task Kids First DRC Somatic Variant Workflow run - BS_HGE47RGT -vs- BS_YZZ7FDBP
Processing vm data
Processing task Kids First DRC Somatic Variant Workflow run - BS_6FVT4ZZJ -vs- BS_B69YME43
Processing vm data
Processing

Processing vm data
Processing task Kids First DRC Somatic Variant Workflow run - BS_9E2FF9ZP -vs- BS_B4JS5CB6
Processing vm data
Processing task Kids First DRC Somatic Variant Workflow run - BS_1KP6JADZ -vs- BS_1X3JQ424
Processing vm data
Processing task Kids First DRC Somatic Variant Workflow run - BS_44ANHKWM -vs- BS_FY5KW796
Processing vm data
Processing task Kids First DRC Somatic Variant Workflow run - BS_1GFA9NF0 -vs- BS_2PVZE2AG
Processing vm data
Processing task Kids First DRC Somatic Variant Workflow run - BS_537YFJ06 -vs- BS_04G6D4W2
Processing vm data
Processing task Kids First DRC Somatic Variant Workflow run - BS_1AMAP96X -vs- BS_E79GK5SS
Processing vm data
Processing task Kids First DRC Somatic Variant Workflow run - BS_97M1BPNE -vs- BS_Q1JQ16CZ
Processing vm data
Processing task Kids First DRC Somatic Variant Workflow run - BS_G3B24JW8 -vs- BS_BDBTPH49
Processing vm data
Processing task Kids First DRC Somatic Variant Workflow run - BS_QE6NEYTV -vs- BS_02MN8B6F
Processing

In [6]:
def get_true_run_time(data):
    data = sorted(data, key=lambda x: x[0])


    # from https://stackoverflow.com/questions/34797525/how-to-correctly-merge-overlapping-datetime-ranges-in-python
    result = []
    try:
        t_old = data[0]
        for t in data[1:]:
            if t_old[1] >= t[0]:  #I assume that the data is sorted already
                t_old = ((min(t_old[0], t[0]), max(t_old[1], t[1])))
            else:
                result.append(t_old)
                t_old = t

        else:
            result.append(t_old)
    except Exception as e:
        print(e)
        pdb.set_trace()
        hold = 1
    total_seconds = 0
    for t_int in result:
        total_seconds += (t_int[1] - t_int[0]).seconds
    return total_seconds/3600


In [10]:
# job_blacklist = ['generate_gvcf', 'gatk_haplotypecaller', 'picard_collectgvcfcallingmetrics', 'checkcontamination', 'picard_mergevcfs']
job_blacklist = []
spot_instance= { 'c4.2xlarge': 0.1896, 'c5.4xlarge': 0.2901, 'c5.9xlarge': 0.6689, 'c4.8xlarge': 0.5653, 'r5.2xlarge': 0.1795, 'c5.2xlarge': 0.1684, 'm5.2xlarge': 0.1405,'m5.xlarge': 0.0686, 'r5.xlarge': 0.0718}
on_demand_instance = { 'c4.2xlarge': 0.398, 'c5.4xlarge': 0.68, 'c5.9xlarge': 1.53, 'c4.8xlarge': 1.591, 'r5.2xlarge': 0.504, 'c5.2xlarge': 0.34, 'm5.2xlarge': 0.384, 'm5.xlarge': 0.192, 'r5.xlarge': 0.252}
#sentieon = '/Users/brownm28/Documents/2022-Feb-24_gtex_gencode_compare/EXPORT/sentieon_estimates.txt'
#fy17 = '/Users/brownm28/Documents/2022-Feb-24_gtex_gencode_compare/EXPORT/fy17_sobreira_estimates.txt'
#process_file(sentieon, '/Users/brownm28/Documents/2022-Feb-24_gtex_gencode_compare/EXPORT/sentieon_summary.txt', job_blacklist, spot_instance, on_demand_instance)
#process_file(fy17, '/Users/brownm28/Documents/2022-Feb-24_gtex_gencode_compare/EXPORT/fy17_sobreira_summary.txt', job_blacklist, spot_instance, on_demand_instance)
somatic_subset = '/Users/brownm28/Documents/playground/somatic_cost_estimates/subset.txt'
process_file(somatic_subset, '/Users/brownm28/Documents/playground/somatic_cost_estimates/subset_summary.txt', job_blacklist, spot_instance, on_demand_instance)

In [ ]:
## Get task log

In [ ]:
tid = '575f8184-f6b1-4191-b119-1672b75d8d61'
project = 'zhangb1/kf-somatic-tools-test'
task = api.tasks.get(tid)
for job in task.get_execution_details().jobs:
    if re.search('seg', job.name):
        print (job.logs['cmd.log'].content())

### Get job run times

In [ ]:
task = api.tasks.get("7174d826-422d-4577-a288-3c6adc1f6f5a")
phrase = "vardict"
for job in task.get_execution_details().jobs:
    if job.status == "COMPLETED" and re.search(phrase, job.name):
        # pdb.set_trace()
        try:
            print (job.name + "\t" + str((job.end_time - job.start_time).seconds/3600))
        except Exception as e:
            print (e)
            pdb.set_trace()
#hold=1

### Get instance run times

In [ ]:
task = api.tasks.get("8acc7912-5380-4db0-8edc-a21ed93833d0")
phrase = "vardict_\d+"
data = {}
for job in task.get_execution_details().jobs:
    if job.status == "COMPLETED" and re.search(phrase, job.name):
        try:
            # print (job.name + "\t" + str((job.end_time - job.start_time).seconds/3600))
            sys.stderr.write(job.name + '\n')
            vm = job.instance.id
            if vm not in data:
                data[vm] = []
            pair = (job.start_time, job.end_time)
            data[vm].append(pair)
        except Exception as e:
            print (e)
            pdb.set_trace()
            hold = 1
total_hrs = 0
c5_9_spot = 0.5164
c5_9_direct = 1.53
m4_10_spot = 0.412
m4_10_direct = 2
spot_rate = c5_9_spot
direct_rate = c5_9_direct
for vm in data:
    data[vm] = sorted(data[vm], key=lambda x: x[0])
    result = []
    try:
        t_old = data[vm][0]
        for t in data[vm][1:]:
            if t_old[1] >= t[0]:  #I assume that the data is sorted already
                t_old = ((min(t_old[0], t[0]), max(t_old[1], t[1])))
            else:
                result.append(t_old)
                t_old = t

        else:
            result.append(t_old)
    except Exception as e:
        print(e)
        pdb.set_trace()
        hold = 1
    total_seconds = 0
    for t_int in result:
        total_seconds += (t_int[1] - t_int[0]).seconds
    #print ('Task ran in ' + str(total_seconds) + ', which is ' + str(total_seconds/3600) + ' hours')
    print(vm + '\t' + '\t' + str(total_seconds/3600))
    total_hrs += total_seconds/3600
print ("All spot\t" + str(total_hrs * spot_rate) + "\t" + str(total_hrs))
print ("All direct\t" + str(total_hrs * direct_rate))
    

## Get task input file IDs

In [ ]:
phrase = "kfdrc-combined-CNV"
project= "kfdrc-harmonization/sd-bhjxbdqk-08"
in_keys = ["b_allele", "input_normal_aligned", "input_tumor_aligned"]
out = open("/Users/brownm28/Documents/playground/cbttc_task_check.txt", "w")
out.write("Task ID\tTask name\tTask status\tb allele bs id\tb allele pt id\tnormal bs id\tnormal pt id\ttumor bs id\ttumor pt id\tpaired vcf N bs id\tpaired vcf T bs id\tpaired vcf pt id\n")
for task in api.tasks.query(project=project).all():
    if re.search(phrase, task.name) and (task.status == "COMPLETED" or task.status == "FAILED"):
        info = []
        try:
            for key in in_keys:
                info.append(task.inputs[key].metadata['Kids First Biospecimen ID'])
                info.append(task.inputs[key].metadata['Kids First Participant ID'])
            info.append(task.inputs['paired_vcf'].metadata['Kids First Biospecimen ID Normal'])
            info.append(task.inputs['paired_vcf'].metadata['Kids First Biospecimen ID Tumor'])
            info.append(task.inputs['paired_vcf'].metadata['Kids First Participant ID'])
            out.write(task.id + "\t" + task.name + "\t" + task.status + "\t" + "\t".join(info) + "\n")
        except Exception as e:
            print (e)
            pdb.set_trace()
            print(task.name + "\t" + task.id + "\tstatus: " + task.status + " failed parsing. skipping!")
out.close()

# Get Task Job err log info

In [ ]:
def get_task_info(task):
    if re.search(phrase, task.name):
        tname = task.name
        tstatus = task.status
        key = task.id + "\t" + tname + "\t" + tstatus
        task_list.append(key)
        return 0
    else:
        return 1

In [ ]:
project = 'kfdrc-harmonization/sd-p445achv-02'
phrase = "Vardict-Somatic"
pwd = "/Users/brownm28/Documents/2019-Nov-4_job_err_log_audit/"
out_fn = 'boyd_vardict_task_info.txt'
print (str(api.remaining) + " remaining tasks")
out = open(pwd + out_fn, 'w')
tasks = api.tasks.query(project=project).all()
task_list = []
valid = 1
n = 50
with concurrent.futures.ThreadPoolExecutor(16) as executor:
    results = {executor.submit(get_task_info, task): task for task in tasks}
    for task_info in concurrent.futures.as_completed(results):
        if (valid % n) == 0:
            sys.stderr.write("Processed " + str(valid) + " valid tasks\n")
            print (api.remaining)
        if task_info.result() == 0:
            valid += 1
# for task in tasks:
#     parse_job_info(task)

out.write("Task ID\tTask Name\tTask Status\n")
for info in task_list:
    out.write(info + "\n")
out.close()


In [ ]:
def parse_bulk_info(task):
    cur_task = task.resource
    tid = cur_task.id
    job_dict[tid] = []
    jphrase = 'vardict_\d+_s'
    for job in cur_task.get_execution_details().jobs:
        if re.search(jphrase, job.name):
            # pdb.set_trace()
            job_dict[tid].append([job.name, job.status, job.logs['job.err.log'].id, str(job.logs['job.err.log'].size)])


In [ ]:
pwd = "/Users/brownm28/Documents/2019-Nov-4_job_err_log_audit/"
manifest_fn = 'boyd_vardict_task_info.txt'
task_manifest = open(pwd + manifest_fn)
head = next(task_manifest)
task_ids = []
for line in task_manifest:
    info = line.rstrip('\n').split('\t')
    task_ids.append(info[0])
task_manifest.close()
task_bulk = []
x = 1
max_j = 50
total = len(task_ids)
for i in range(0, total, max_j):
    uset = i + max_j
    if uset > total:
        uset = total
    task_bulk.extend(api.tasks.bulk_get(tasks=task_ids[i:uset]))
project = 'kfdrc-harmonization/sd-p445achv-02'
phrase = "Vardict-Somatic"

out_fn = 'boyd_vardict_job_info.txt'
out = open(pwd + out_fn, 'w')
out.write("Task ID\tJob Name\tJob Status\tError log ID\tError log size\n")
set_n = 1
for i in range(0, total, max_j):
    uset = i + max_j
    if uset > total:
        uset = total
    job_dict = {}
    sys.stderr.write("Processing set " + str(set_n) + ", api calls remaining: " + str(api.remaining) + "\n")
#     for task in task_bulk[i:uset]:
#         parse_bulk_info(task)
    with concurrent.futures.ThreadPoolExecutor(8) as executor:
        results = {executor.submit(parse_bulk_info, task): task for task in task_bulk[i:uset]}
    try:
        for tid in job_dict:
            for job in job_dict[tid]:
                out.write(tid + "\t" + "\t".join(job) + "\n")
        out.flush()
        sys.stderr.write("Finished with set " + str(set_n) + ", api calls remaining: " + str(api.remaining) + '\n')
        set_n += 1
    except Exception as e:
        sys.stderr.write(str(e) + "\n")
        pdb.set_trace()
        hold = 1

out.close()

    

In [ ]:
task_list = ['72e5103a-4491-4073-a615-cd830ec63690', 'fe5aeb6d-a72d-4319-b8de-12083684376e']
task_bulk = api.tasks.bulk_get(tasks=task_list)
pdb.set_trace()
pause = 1

## Get wf used in task

In [ ]:
pwd = "/Users/brownm28/Documents/2019-Nov-4_job_err_log_audit/to_check/"
manifest_fn = 'pbta.txt'
task_manifest = open(pwd + manifest_fn)
out_fn = 'pbta_vardict_app.txt'
out = open(pwd + out_fn, 'w')
head = next(task_manifest)
for line in task_manifest:
    info = line.rstrip('\n').split('\t')
    task = api.tasks.get(info[0])
    out.write(task.id + "\t" + task.app + "\n")
out.close()

## Fix swapped inputs

In [ ]:
def check_fix_swap(task):
    try:
        if re.search('LANCET_TCGA', task.name):
            if not re.search('-10A-', task.inputs['input_normal_aligned'].name):
                sys.stderr.write('Inputs swapped, re-tasking ' + task.name + '\n')
                task_name = 'LANCET: ' + task.inputs['input_tumor_name'] + " " + task.inputs['input_normal_name']
                in_dict = {}
                for key in task.inputs:
                    in_dict[key] = task.inputs[key]
                in_dict['input_tumor_aligned'] = task.inputs['input_normal_aligned']
                in_dict['input_normal_aligned'] = task.inputs['input_tumor_aligned']
                task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=in_dict, run=False)
                task.inputs['output_basename'] = task.id
                task.save()
    except Exception as e:
        sys.stderr.write(str(e) + "\nProcessing " + task.name + " failed\n")
        #pdb.set_trace()
        hold = 1
        sys.exit()
        

In [ ]:
project = 'cavatica/openpbta-tcga'
app_name = project + "/kfdrc-lancet-wf-baminput"
tasks = api.tasks.query(project=project, status="COMPLETED").all()
i = 1
n = 50

with concurrent.futures.ThreadPoolExecutor(8) as executor:
    results = {executor.submit(check_fix_swap, task): task for task in tasks}
    for result in concurrent.futures.as_completed(results):
        if i % n == 0:
            sys.stderr.write(str(i) + ' tasks set up\n')
        i += 1
# for task in tasks:
#     check_fix_swap(task)

### Re-draft failed task in another project

In [ ]:
old_task = '416aea5b-0722-45db-9b40-8a58ed685769'
old_task_info = api.tasks.get(old_task)
project = 'zhangb1/kf-somatic-tools-test'
app_name = project + "/kfdrc-combined-somatic-wgs-cnv-wf"

in_dict = {}
for key in old_task_info.inputs:
    in_dict[key] = old_task_info.inputs[key]
    try:
        in_dict[key].copy(project=project)
    except Exception as e:
        sys.stderr.write(str(e) + "\n")
in_dict['output_basename'] = 'theta2_fail_catch_test'
task_name = "Theta2 Fail Catch Test"
task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=in_dict, run=False)
task.save()

In [ ]:
task_id = 'fa9f548e-5dfb-4562-a940-8c81cb5a428b'
task_info = api.tasks.get(task_id)
pdb.set_trace()
hold=1

# Stupid folder del because sbg won't help

In [3]:
project= 'd3b-bixu/kf-amplicon-architect-dev'
files = api.files.query(project=project).all()
dir_list = []
for file in files:
    if file.is_folder() and re.search('_hg38_ref_cache', file.name):
        dir_list.append(file)

In [4]:
def delete_folder_contents(dir_obj):
    for item in dir_obj.list_files().all():
        if item.is_folder():
            dir_lvl.append(item)
            delete_folder_contents(item)
        else:
            item.delete()


In [5]:
for i in range(len(dir_list)):
    print("Deleting " + dir_list[i].name)
    dir_lvl = []
    delete_folder_contents(dir_list[i])
    for item in dir_lvl[::-1]:
        item.delete()     

Deleting _11_hg38_ref_cache
Deleting _12_hg38_ref_cache
Deleting _15_hg38_ref_cache
Deleting _17_hg38_ref_cache
Deleting _18_hg38_ref_cache
Deleting _19_hg38_ref_cache
Deleting _1_hg38_ref_cache
Deleting _21_hg38_ref_cache
Deleting _22_hg38_ref_cache
Deleting _23_hg38_ref_cache
Deleting _24_hg38_ref_cache
Deleting _25_hg38_ref_cache
Deleting _26_hg38_ref_cache
Deleting _27_hg38_ref_cache
Deleting _28_hg38_ref_cache
Deleting _29_hg38_ref_cache
Deleting _2_hg38_ref_cache
Deleting _30_hg38_ref_cache
Deleting _31_hg38_ref_cache
Deleting _32_hg38_ref_cache
Deleting _33_hg38_ref_cache
Deleting _34_hg38_ref_cache
Deleting _35_hg38_ref_cache
Deleting _36_hg38_ref_cache
Deleting _37_hg38_ref_cache
Deleting _38_hg38_ref_cache
Deleting _39_hg38_ref_cache
Deleting _3_hg38_ref_cache
Deleting _40_hg38_ref_cache
Deleting _41_hg38_ref_cache
Deleting _42_hg38_ref_cache
Deleting _43_hg38_ref_cache
Deleting _44_hg38_ref_cache
Deleting _45_hg38_ref_cache
Deleting _46_hg38_ref_cache
Deleting _47_hg38_ref_c

# Get job mem usage
Niche use case to parse log output file and collate usage

In [15]:
def parse_task(task):
    # edit this to get what job name you want info on
    phrase = ".*model_segments.*"

    for job in task.get_execution_details().jobs:
        if job.status == "COMPLETED" and re.search(phrase, job.name):
            if job.name not in mem_dict:
                mem_dict[job.name] = []
            log_data = job.logs['stderr'].content()
            mem_info = re.findall("Runtime.totalMemory\(\)=(\d+)", log_data)
            # convert to GB
            mem_in_gb = round(int(mem_info[0])/(1024*1024*1024))
            mem_dict[job.name].append(mem_in_gb)


In [16]:
project = "kfdrc-harmonization/sd-bhjxbdqk-x01-somatic-mutations-wgs-tumor"
t_phrase = "Kids First DRC Somatic Variant Workflow"
tasks = api.tasks.query(project=project, status="COMPLETED").all()
mem_dict = {}
for task in tasks:
    if re.search(t_phrase, task.name):
        parse_task(task)

In [17]:
results = open("/Users/brownm28/Documents/playground/gatk_cnv_model_mem_usage.txt", "w")
results.write("job_name\tmem_usage_GB\n")
for job in mem_dict:
    for size in mem_dict[job]:
        results.write(job + "\t" + str(size) + "\n")
results.close()